# Model Klasifikasi Baju Layak dan Tidak Layak Menggunakan CNN Tanpa Pretraining

In [1]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 933.6 kB/s eta 0:00:00


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Define paths
train_dir = '/content/drive/MyDrive/Bangkit/data_split/train'
val_dir = '/content/drive/MyDrive/Bangkit/data_split/val'

## Preprocessing Data

In [9]:
def train_val_generators(train_dir, val_dir):

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary'
    )

    return train_generator, validation_generator

In [10]:
train_generator, validation_generator = train_val_generators(train_dir, val_dir)

Found 812 images belonging to 2 classes.
Found 374 images belonging to 2 classes.


In [ ]:

# print("Sample happy image:")
# plt.imshow(load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}"))
# plt.show()

# print("\nSample sad image:")
# plt.imshow(load_img(f"{os.path.join(sad_dir, os.listdir(sad_dir)[0])}"))
# plt.show()

## Membuat Model Tanpa Pretraining

In [12]:
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),  # Add dropout for regularization
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])


    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])


## Melatih Model

In [13]:
  # Add custom callback to stop after accuracy > 99%
  class EarlyStoppingByAccuracy(Callback):
      def on_epoch_end(self, epoch, logs=None):
          accuracy = logs.get("accuracy")
          if accuracy is not None and accuracy > 0.99:
              print(f"\nAccuracy reached {accuracy * 100:.2f}%, stopping training.")
              self.model.stop_training = True

In [14]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min'),
    EarlyStoppingByAccuracy()
]

In [15]:

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50,
    callbacks=callbacks
)


Epoch 1/50
25/25 [==============================] - 293s 12s/step - loss: 0.9159 - accuracy: 0.5308 - val_loss: 0.7017 - val_accuracy: 0.4631 - lr: 0.0010
Epoch 2/50
25/25 [==============================] - 18s 697ms/step - loss: 0.6952 - accuracy: 0.5359 - val_loss: 0.6929 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 3/50
25/25 [==============================] - 17s 679ms/step - loss: 0.6932 - accuracy: 0.5205 - val_loss: 0.6937 - val_accuracy: 0.4631 - lr: 0.0010
Epoch 4/50
25/25 [==============================] - 19s 764ms/step - loss: 0.6927 - accuracy: 0.5231 - val_loss: 0.6940 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 5/50
25/25 [==============================] - 19s 739ms/step - loss: 0.6975 - accuracy: 0.5475 - val_loss: 0.6846 - val_accuracy: 0.5369 - lr: 0.0010
Epoch 6/50
25/25 [==============================] - 17s 682ms/step - loss: 0.6962 - accuracy: 0.5333 - val_loss: 0.6912 - val_accuracy: 0.5199 - lr: 0.0010
Epoch 7/50
25/25 [==============================] - 18s 709ms/ste

In [16]:
loss_model, accuracy_model = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f"Model Validation Accuracy: {accuracy_model * 100:.2f}%")

11/11 [==============================] - 2s 150ms/step - loss: 0.4057 - accuracy: 0.8352
Model Validation Accuracy: 83.52%
